In [1]:
import torch
import transformers
import tiktoken
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch.nn.functional as F

# Goal is to take a sequence of tokens, and return the topk tokens at position i
# Such that the tokens maximize the negative gradient of the loss function across the entire sequence

# Initial sequence

def setup_model_and_tokenizer():
    # Initialize model and tokenizer
    model_name = "lmsys/vicuna-7b-v1.5"
    chat_template = "{% if messages[0]['role'] == 'system' %}{% set loop_messages = messages[1:] %}{% set system_message = messages[0]['content'] %}{% else %}{% set loop_messages = messages %}{% set system_message = 'A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user\\'s questions.' %}{% endif %}{% for message in loop_messages %}{% if (message['role'] == 'user') != (loop.index0 % 2 == 0) %}{{ raise_exception('Conversation roles must alternate user/assistant/user/assistant/...') }}{% endif %}{% if loop.index0 == 0 %}{{ system_message }}{% endif %}{% if message['role'] == 'user' %}{{ ' USER: ' + message['content'].strip() }}{% elif message['role'] == 'assistant' %}{{ ' ASSISTANT: ' + message['content'].strip() + eos_token }}{% endif %}{% endfor %}{% if add_generation_prompt %}{{ ' ASSISTANT:' }}{% endif %}"
    tokenizer = AutoTokenizer.from_pretrained(model_name, chat_template=chat_template)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16,
        device_map="auto"
    )
    return model, tokenizer



In [3]:
# Setup
model, tokenizer = setup_model_and_tokenizer()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/ubuntu/dspy-propane/.venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/ubuntu/dspy-propane/.venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/ubuntu/dspy-propane/.venv/li

In [8]:
def get_gradient_tokens(model, tokenizer, sequence, position, top_k=5):
    # Tokenize the input sequence
    messages = [{"role": "user", "content": sequence}]
    inputs = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt")
    input_ids = inputs.to(model.device)
    
    # Create a copy of input_ids that requires gradient
    input_embeds = model.get_input_embeddings()(input_ids)
    input_embeds.requires_grad_(True)
    input_embeds.retain_grad()
    
    # Forward pass
    outputs = model(inputs_embeds=input_embeds)
    logits = outputs.logits
    
    # Calculate loss (we'll use the current token prediction loss)
    # Don't shift the logits/labels since we want current token prediction
    loss = F.cross_entropy(logits.view(-1, logits.size(-1)),
                          input_ids.view(-1))
    
    # Calculate gradients
    loss.backward()
    
    # Get gradients at the specified position
    position_gradients = input_embeds.grad[0, position]
    
    # Get the token embeddings
    token_embeddings = model.get_input_embeddings().weight
    
    # Calculate similarity between position gradients and all token embeddings
    similarities = torch.matmul(token_embeddings, position_gradients)
    
    # Get top-k tokens that maximize the negative gradient
    top_values, top_indices = torch.topk(-similarities, k=top_k)
    
    # Convert token ids to tokens
    top_tokens = [tokenizer.decode([idx.item()]) for idx in top_indices]
    
    return top_tokens, list(map(lambda x: -x, top_values.tolist()))

In [10]:

sequence = "What is the capital of France?"

print(f"Analyzing sequence: {sequence}")

print("Converting to chatml format")
messages = [{"role": "user", "content": sequence}]
inputs = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True, return_tensors="pt")
print(inputs)
# Get tokenized sequence length
tokens = tokenizer.encode(inputs, return_tensors="pt")[0]
seq_length = len(tokens)

print(f"\nTokenized sequence length: {seq_length}")
print("Analyzing gradient-based importance for each position...")
# only change the tokens that are a part of the prompt
prompt_tokens = tokenizer.encode(sequence, return_tensors="pt")[0]
# Analyze each position
for i in range(seq_length - 1):
    tokens_at_pos = tokenizer.decode([tokens[i]])
    print(f"\nPosition {i} (current token: '{tokens_at_pos}')")
    
    top_tokens, importance_scores = get_gradient_tokens(model, tokenizer, sequence, i)
    
    print("Top replacement tokens by gradient magnitude:")
    for token, score in zip(top_tokens, importance_scores):
        print(f"  Token: '{token}', Score: {score:.4f}")

Analyzing sequence: What is the capital of France?
Converting to chatml format
A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. USER: What is the capital of France? ASSISTANT:

Tokenized sequence length: 46
Analyzing gradient-based importance for each position...

Position 0 (current token: '<s>')
Top replacement tokens by gradient magnitude:
  Token: 'A', Score: -0.6602
  Token: '
', Score: -0.2008
  Token: 'А', Score: -0.1494
  Token: 'A', Score: -0.1008
  Token: 'The', Score: -0.0703

Position 1 (current token: 'A')
Top replacement tokens by gradient magnitude:
  Token: 'disag', Score: -0.0797
  Token: 'indic', Score: -0.0774
  Token: 'compet', Score: -0.0681
  Token: 'autonom', Score: -0.0680
  Token: 'kilom', Score: -0.0672

Position 2 (current token: 'chat')
Top replacement tokens by gradient magnitude:
  Token: 'eigh', Score: -0.0774
  Token: 'боль', Score: -0.0742
  Token: 

IndexError: index 45 is out of bounds for dimension 1 with size 45